# Automate Data Collection of Attributes with Selenium in Python

Using Selenium WebDriver to webscrape on Chrome. I specifically used Selenium to retrieve content loaded dynamically with JS on vocabulary.com.

Goal: compile a complete list of adverbs and adjectives from a dictionary website, load into a dataframe, and export to CSV.

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from collections import defaultdict
import csv
import pandas as pd


Fire up the Webdriver to open the destination webpage: vocabulary.com's dictionary page 

In [ ]:
chrome_path = r"C:\Users\annam\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(chrome_path)
driver.get('https://www.vocabulary.com/dictionary/')
print(driver.title)

Vocabulary.com Dictionary - Meanings, Definitions, Quizzes, and Word Games : Vocabulary.com


Enter in lookup terms for adjectives and adverbs, simulating advanced search submission

In [ ]:
wait = WebDriverWait(driver,10)
wait.until(EC.element_to_be_clickable((By.ID,"search")))
search_box = driver.find_element_by_id("search")
search_box.send_keys('pos:adjective',Keys.RETURN)

wait2 = WebDriverWait(driver,10)
wait2.until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"entry")))
terms = driver.find_elements_by_class_name("entry")

In [ ]:
# example code
auto = driver.find_element(By.ID, "autocomplete")
wait = WebDriverWait(driver,10)

driver.execute_script("arguments[0].scrollIntoView()", auto)
driver.execute_script("arguments[0].scrollIntoView(true)", terms[19])
wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"entry")))
terms = driver.find_elements_by_class_name("entry")
driver.execute_script("arguments[0].scrollIntoView()", auto)
print(terms[20].text)
driver.execute_script("arguments[0].scrollIntoView()", auto)

driver.execute_script("arguments[0].scrollIntoView(true)", terms[39])
wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"entry")))
terms = driver.find_elements_by_class_name("entry")
driver.execute_script("arguments[0].scrollIntoView()", auto)
print(terms[40].text)
driver.execute_script("arguments[0].scrollIntoView()", auto)

# print([terms[i].text for i in range(len(terms))])

115 being five more than one hundred ten
14th coming next after the thirteenth in position


In [ ]:
# scroll to end of vocab list
def scrollToBottom(terms):
    auto = driver.find_element(By.ID, "autocomplete")
    wait = WebDriverWait(driver,10)
    canScroll = True # when len of terms remains the same...stop
    term_id = 19
    tries = 3
    while canScroll:
        for i in range(tries):
            try:
                if term_id < 1000: #len(terms):
                    driver.execute_script("arguments[0].scrollIntoView()", auto)
                    driver.execute_script("arguments[0].scrollIntoView(true)", terms[term_id])

                    wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"entry")))
                    terms = driver.find_elements_by_class_name("entry")

                    driver.execute_script("arguments[0].scrollIntoView()", auto)
                    # print(terms[term_id+1].text) # for debugging
                    driver.execute_script("arguments[0].scrollIntoView()", auto)

                    term_id += 20
                else: canScroll = False
                break
            except:
                # pass
                driver.execute_script("arguments[0].scrollIntoView()", auto)
                driver.execute_script("arguments[0].scrollIntoView(true)", terms[term_id])
                driver.execute_script("arguments[0].scrollIntoView()", auto)
                wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"entry")))
                terms = driver.find_elements_by_class_name("entry")
                driver.execute_script("arguments[0].scrollIntoView()", auto)
                print('exception at ' + str(term_id))
        else:
            # enter here only if we have failed all our tries, break out of while loop
            print("broken at " + str(term_id))
            term_id = 1000 # canScroll = False
    print('done at ' + str(term_id))
    return terms


terms = driver.find_elements_by_class_name("entry")
terms = scrollToBottom(terms)

done at 1019


In [ ]:
print(len(terms))
words = []
defs = []
def iterwords(terms):
    for term in terms:
        word = term.find_element_by_class_name("word").text
        words.append(word)
        defin = term.find_element_by_class_name("definition").text
        defs.append(defin)

iterwords(terms)
print(words)
# print(defs)

1020
['in', 'i', 'on', 'all', '0', '1', '10', '100', '1000', '1000th', '100th', '101', '101st', '105', '105th', '10-membered', '10th', '11', '110', '110th', '115', '115th', '11th', '12', '120', '120th', '125', '125th', '12th', '13', '130', '130th', '135', '135th', '13th', '14', '140', '140th', '145', '145th', '14th', '15', '150', '150th', '155', '155th', '15th', '16', '160', '160th', '165', '165th', '16th', '17', '170', '170th', '175', '175th', '17th', '18', '180', '180th', '18th', '19', '190', '190th', '19th', '1st', '2', '20', '200', '200th', '20th', '21', '21st', '22', '.22 caliber', '.22-caliber', '.22 calibre', '.22-calibre', '22nd', '23', '23rd', '24', '24th', '25', '25th', '26', '26th', '27', '27th', '28', '28th', '29', '29th', '2d', '2-dimensional', '2nd', '3', '30', '300', '300th', '30th', '31', '31st', '32', '32nd', '33', '33rd', '34', '34th', '35', '35th', '36', '36th', '37', '37th', '38', '.38 caliber', '.38-caliber', '.38 calibre', '.38-calibre', '38th', '39', '39th', '3-d

In [ ]:
words.index('abactinal') #236

In [ ]:
search_box.clear()

In [ ]:
driver.quit()

Load word data onto dataframe

In [ ]:
data = defaultdict(list)
words_ids = list(range(0,len(terms),1))
for i in range(len(terms)):
    data['Id'].append(i+1)
    data['Attribute'].append(words[i])
    data['Definition'].append(defs[i])

df = pd.DataFrame(data)
df

,Id,Attribute,Definition
0,1,in,directed or bound inward
1,2,i,used of a single unit or thing; not two or more
2,3,on,in operation or operational
3,4,all,quantifier
4,5,0,indicating the absence of any or all units und...
...,...,...,...
1015,1016,amebic,pertaining to or resembling amoebae
1016,1017,ameboid,like an amoeba
1017,1018,amebous,pertaining to or resembling amoebae
1018,1019,ameliorating,tending to ameliorate


In [ ]:
df.tail()

,Id,Attribute,Definition
1015,1016,amebic,pertaining to or resembling amoebae
1016,1017,ameboid,like an amoeba
1017,1018,amebous,pertaining to or resembling amoebae
1018,1019,ameliorating,tending to ameliorate
1019,1020,ameliorative,tending to ameliorate


In [ ]:
# save the dataframe as csv
df.to_csv('vocab_data.csv', index=False)

Find synonyms and antonyms of each word on vocabulary.com advanced search